In [180]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from RNN_block import RNN_block
import pennylane as qml
import pandas as pd


In [3]:
df = pd.read_csv('./dataset.csv')

In [323]:
n_train = 250

y_train = torch.tensor(df['nat_demand'].to_numpy()[:n_train]).to(torch.float)
x_train = torch.tensor(df[sorted("T2M_toc	QV2M_toc TQL_toc W2M_toc T2M_san QV2M_san TQL_san W2M_san T2M_dav QV2M_dav TQL_dav W2M_dav".split())].to_numpy()[:n_train]).to(torch.float)

xs_train = torch.tensor([(x_train[:,3 * i : 3 * i + 3].sum(dim=1)/3).tolist() for i in range(4)]).T
print(xs_train[1])
y_train = (y_train-y_train.min())/(y_train.max()-y_train.min())




tensor([1.7476e-02, 2.3959e+01, 3.5131e-02, 1.2807e+01])


In [324]:
import pennylane as qml
class RNN_block:
    def __init__(self, input_size):
        self.input_size = input_size

    
    def embedding(self, params):
        n = (self.input_size + 1) // 2
        for i in range(n):
            qml.Hadamard(i)
            qml.RZ(2.0 * params[:,i], i)
        
        for i in range(n - 1):
            qml.IsingZZ(2.0 * params[:,n + i] ,[i , i + 1])
    
    def ansatz(self, params, all_entangled = False):
        # Length of Params : 3 * num_qubit
        n = self.input_size
        for i in range(n):
            qml.RX(params[3 * i], i)
            qml.RY(params[3 * i + 1], i)
            qml.RZ(params[3 * i + 2], i)
        for i in range(n - 1):
            qml.CNOT([i, i + 1])
        if all_entangled:
            qml.CNOT([n - 1, 0])

In [325]:
class G1(nn.Module):
    '''
    G for V(G)
    '''
    def __init__(self, input):
        super(G1, self).__init__()
        self.li1 = nn.Linear(input,16)
        self.li2 = nn.Linear(16, 16)
        self.li3 = nn.Linear(16, 2*input-1)
        
    def forward(self, x):
        x = self.li1(x)
        x = F.relu(x)
        x = self.li2(x)
        x = F.relu(x)
        x = self.li3(x)
        x = 2*torch.pi*F.relu(x)
        return x

In [326]:
n_qu = 4
dev = qml.device("default.qubit", wires = n_qu)

def embedding(params, n_qu):
    '''
    embedding layer
    '''
    n = n_qu
    for i in range(n):
        qml.Hadamard(i)
        qml.RZ(2.0 * params[:,i], i)
    
    for i in range(n - 1):
        qml.IsingZZ(2.0 * params[:,n + i] ,[i , i + 1])

@qml.qnode(dev, interface="torch")
def fidelity(vec1, vec2, n_qu):
    '''
        Args:
            vec1 : list, (2n - 1)개의 element로 이루어진 vector
            vec2 : list, (2n - 1)개의 element로 이루어진 vector
    '''
    embedding(vec1, n_qu) # Phi(x1) circuit 적용
    qml.adjoint(embedding)(vec2, n_qu) # Phi^t(x2) 적용
    return qml.probs()


In [399]:
class feature_train(nn.Module):
    def __init__(self,n_qu):
        super(feature_train, self).__init__()
        self.n_qu = n_qu
        self.linear1 = G1(input=n_qu)
        self.quantum_layer = fidelity
    def forward(self,inputs):
        input1 = inputs[0]
        input2 = inputs[1]
        input1 = self.linear1(input1)
        input2 = self.linear1(input2)
        output = self.quantum_layer(input1,input2,self.n_qu)[:,0]
        return output

import torch
from torch.utils.data import DataLoader, TensorDataset, random_split     

class data_seq():
    def __init__(self,feature_data,label_data):
        self.feature = feature_data
        self.label = label_data
    
    def split_data(self,test_ratio=0.2, batch_size=32,seq_first = False):
        """
        데이터를 훈련 및 테스트 세트로 나누고 배치 단위로 나누어 줍니다.

        Args:
            features (torch.Tensor): 입력 특징 텐서
            labels (torch.Tensor): 레이블 텐서
            test_ratio (float): 테스트 세트 비율 (기본값: 0.2)
            batch_size (int): 배치 크기 (기본값: 32)

        Returns:
            train_loader (DataLoader): 훈련 데이터 로더
            test_loader (DataLoader): 테스트 데이터 로더
        """
        if seq_first:
            self.feature = self.feature.permute(1,0,2)
            self.label = self.label.permute(1,0)
        
        # 데이터셋 생성
        dataset = TensorDataset(self.feature, self.label)

        # 훈련 및 테스트 데이터셋 크기 계산
        test_size = int(len(dataset) * test_ratio)
        train_size = len(dataset) - test_size

        # 데이터셋 분할
        train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

        # 데이터 로더 생성
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        return train_loader, test_loader

class train_seq():
    def __init__(self,model,train_loader,test_loader):
        self.model= model
        self.train_data = train_loader
        self.test_data = test_loader
    def train(self,epochs,optimizer,criterion,seq_first = False):
        for epoch in range(epochs):
            pred_list = []
            label_list = []
            for train,label in self.train_data:
                if seq_first:
                    train = train.permute(1,0,2)
                    label = label.permute(0,1)
                
                optimizer.zero_grad()
                pred = self.model(train)
                pred_list.append(pred)
                label_list.append(label)
                loss = criterion(pred,label)
                loss.backward()
                optimizer.step()
            pred = torch.concat(pred_list)
            label = torch.concat(label_list)
            loss = criterion(pred,label)
            loss_test = self.test(criterion,seq_first)
            print(f'epoch : {epoch+1} loss :{loss} loss_test = {loss_test}')
    def test(self,criterion,seq_first = False):
        pred_list = []
        label_list = []
        for test,label in self.test_data:
            if seq_first:
                test = test.permute(1,0,2)
                label = label.permute(0,1)
            pred = self.model(test)
            pred_list.append(pred)
            label_list.append(label)
        pred = torch.concat(pred_list)
        label = torch.concat(label_list)
        loss = criterion(pred,label)
        return loss
        
                
        
                

In [400]:
train_list = []
train_label_list = []
test_list = []
test_label_list = []

for i in range(n_train-1):
    train_data = torch.stack([xs_train,torch.concat([xs_train[(i+1):],xs_train[:(i+1)]])])
    train_list.append(train_data)
    label_data = torch.stack([y_train,torch.concat([y_train[(i+1):],y_train[:(i+1)]])])
    train_label_list.append(label_data)

In [401]:
train_data = torch.concat(train_list,dim=1)
train_label = torch.concat(train_label_list,dim=1)

In [410]:
from torch import optim
pretrain_data = data_seq(train_data,train_label)
train_loader, test_loader = pretrain_data.split_data(batch_size=64,seq_first=True)
model = feature_train(n_qu)
optimizer = optim.Adam(model.parameters(),lr=0.005)

def criterion(pred,label):
    return -torch.sum(torch.log(1-pred)*(label[:,0]-label[:,1])**2 + torch.log(pred)*(1-(label[:,0]-label[:,1])**2))/len(pred)
    

In [411]:
pretrain_seq = train_seq(model,train_loader,test_loader)
pretrain_seq.train(3,optimizer,criterion,seq_first=True)

epoch : 0 loss :1.5986825227737427 loss_test = 1.4027186632156372
epoch : 1 loss :1.3776195049285889 loss_test = 1.3845701217651367
epoch : 2 loss :1.3815345764160156 loss_test = 1.3753087520599365


In [405]:
block = RNN_block(4)
@qml.qnode(dev, interface="torch")
def ansatz(inputs,weights):
    '''
        Args:
            vec1 : list, (2n - 1)개의 element로 이루어진 vector
            vec2 : list, (2n - 1)개의 element로 이루어진 vector
    '''
    block.embedding(inputs)
    block.ansatz(weights) # Phi(x1) circuit 적용
    return qml.probs()

In [406]:
block = RNN_block(4)
class train_model(nn.Module):
    def __init__(self,feature_model):
        super(train_model, self).__init__()
        self.QNE_layer = feature_model.linear1
        self.ansatz = ansatz
        self.Q_param = nn.Parameter(torch.rand([12]).float(),requires_grad=True)
    def forward(self,input):
        input = self.QNE_layer(input)
        output = self.ansatz(input,self.Q_param)[:,3]
        return output
        

In [414]:
data = data_seq(xs_train,y_train)
train_loader,test_loader = data.split_data(batch_size=64)

In [417]:
CLS_model  = train_model(model)
optimizer = optim.Adam([CLS_model.Q_param],lr=0.04)
criterion = nn.MSELoss()

reg_seq = train_seq(model,train_loader,test_loader)

In [418]:
reg_seq.train(5,optimizer,criterion)

IndexError: too many indices for tensor of dimension 1